# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [11]:
import pandas as pd 
import numpy as np 
import csv
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

For Hyperparameter tuning, we will be reusing the same data used in the AutoML script (salary.csv). More accurately, a cleaned version of the dataset is imported into the training file (hp_train.py) to be used for hyperdrive. 

## Configure Experiment

In [71]:
ws = Workspace.from_config()
experiment_name = 'salary-hyperdrive-experiment'

experiment=Experiment(ws, experiment_name)

## Configure Cluster

In [72]:
# Reuse same cluster from AutoML run:
amlcompute_cluster_name = "auto-ml" 

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

Found existing cluster, use it.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [86]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# --> Not required as using Bayesian Sampling

#TODO: Create the different params that you will be using during training
# Specify parameter sampler
ps = BayesianParameterSampling(
    {
        "C": uniform(0.01, 2.0), #C > 2.0 results in 3 min or longer runs 
        "kernel": choice('rbf', 'linear', 'poly', 'sigmoid'),
        "degree": choice(1,2,3,4,5,6,7,8)
    }
)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yaml')


#TODO: Set-up estimator
est = ScriptRunConfig(source_directory='.',
                      script = 'hp_train.py',
                      compute_target=compute_target,
                      environment=sklearn_env
)

#TODO: Create your estimator and hyperdrive config
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=80,
                                     max_concurrent_runs=4)

In [87]:
#TODO: Submit your experiment
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(config=hyperdrive_config)

In [88]:
RunDetails(hyperdrive_run).show()
# (suppressed output due to timeout messages)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

2023-01-28 14:53:07,474 WARNING Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7ff350266c10>, 'Connection to uksouth.experiments.azureml.net timed out. (connect timeout=100)')': /history/v1.0/subscriptions/36cd5b29-0f0b-4505-8ed0-2c1f142e8de3/resourceGroups/mlops-udacity-rg/providers/Microsoft.MachineLearningServices/workspaces/mlops-udacity-ws/experiments/salary-automl-experiment/metrics:query?mergestrategytype=None
2023-01-28 14:54:49,163 WARNING Retrying (Retry(total=1, connect=1, read=3, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7ff350184340>, 'Connection to uksouth.experiments.azureml.net timed out. (connect timeout=100)')': /history/v1.0/subscriptions/36cd5b29-0f0b-4505-8ed0-2c1f142e8de3/resourceGroups/mlops-udacity-rg/providers/Microsoft.MachineLearningServices/workspaces/mlops

In [89]:
hyperdrive_run.wait_for_completion()

{'runId': 'HD_4c5d70d0-ada1-4d26-a7e6-822f635dde0a',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2023-01-28T13:56:31.563868Z',
 'endTimeUtc': '2023-01-28T15:49:12.765456Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9f73f6d3-5f49-4da9-9267-1915a272bb02',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': 'infinite_space_size',
  'score': '0.8170986365311387',
  'best_child_run_id': 'HD_4c5d70d0-ada1-4d26-a7e6-822f635dde0a_63',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_4c5d70d0-ada1-4d26-a7e6-822f635dde0a_63'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

Note to reviewer: The RunDetails widget times out due to turning off compute instance. Therefore I call the RunDetails again using the completed run's ID. 

In [2]:
#Call the previously ran Experiment 
# Required Imports in case of cleared variable workspace:
from azureml.widgets import RunDetails
from azureml.core import Workspace
from azureml.core import Run
from azureml.core import Experiment

ws = Workspace.from_config()

prev_exp = Experiment(ws, name="salary-hyperdrive-experiment")
prev_run = Run(prev_exp, run_id="HD_4c5d70d0-ada1-4d26-a7e6-822f635dde0a")

RunDetails(prev_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [90]:
best_hd_run = hyperdrive_run.get_best_run_by_primary_metric()

print(best_hd_run)

print('\n')

best_run_metrics = best_hd_run.get_metrics()
print(best_run_metrics)

Run(Experiment: salary-hyperdrive-experiment,
Id: HD_4c5d70d0-ada1-4d26-a7e6-822f635dde0a_63,
Type: azureml.scriptrun,
Status: Completed)


{'Regularization Parameter:': 2.0, 'Kernel:': 'poly', 'Degree:': 7, 'Accuracy': 0.8170986365311387}


In [91]:
#TODO: Save the best model

model = best_hd_run.register_model(model_name = 'hyperdrive_salary_model.pkl', model_path='.')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

------

In comparing accuracy (the training metric used for both AutoML and HyperDrive), the HypderDrive model only achieves an accuracy of 81.7% while the AutoML model has an accuracy of 88.6%, therefore only the AutoML model is deployed and this script is complete. 

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

